In [ ]:
import pandas as pd
import re
from collections import Counter
from tqdm import tqdm_notebook as tqdm

In [ ]:
correct_df = pd.read_csv('../Correct_cities.csv')
incorrect_df = pd.read_csv('../Misspelt_cities.csv')

In [ ]:
unique_letters = []
for word in correct_df['name']:
    letters = list(word)
    for l in letters:
        if l not in unique_letters:
            unique_letters.append(l)
for word in incorrect_df['misspelt_name']:
    letters = list(word)
    for l in letters:
        if l not in unique_letters:
            unique_letters.append(l)

In [ ]:
# sorted(unique_letters)

In [ ]:
possible_letters = ''.join(sorted(unique_letters))[8:-2]
possible_letters, len(possible_letters)

In [ ]:
correct_words_list = []
for word in correct_df['name']:
    correct_words_list.append(str(word))

In [ ]:
# building country-wise dictonary
country_dict = {}
correct_sorted = correct_df.sort_values(by='country')
for i in correct_sorted.groupby('country'):
#     print(i[1])
    temp_df = i[1]
    country_dict[i[0]] = list(temp_df['name'])
#     break



In [ ]:
country_dict

In [ ]:

# def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(correct_words_list)

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])#or known(edits3(word))

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = ' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    
    deletes    = [L + R[1:]               for L, R in splits if R]

    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    
    inserts    = [L + c + R               for L, R in splits for c in letters]

    return set(deletes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
def edits3(word):
    return (e3 for e1 in edits1(word) for e2 in edits1(e1) for e3 in edits1(e2))

In [ ]:
correction('PalqyanlCity')

In [ ]:
all_ids = []
all_corrected_names = []
for i in tqdm(range(incorrect_df.shape[0])):
    incorrect_name = incorrect_df.iloc[i]['misspelt_name']
#     print(incorrect_name)
    incorrect_country = incorrect_df.iloc[i]['country']
#     print(incorrect_country)
    corrected = correction(incorrect_name)
    if  corrected in country_dict[incorrect_country]:
        correct_city_name = corrected
        correct_country_df = correct_df[correct_df['country']==incorrect_country]
        correct_city_id = list(correct_country_df[correct_country_df['name']==correct_city_name]['id'])[0]
        all_ids.append(correct_city_id)
        all_corrected_names.append(correct_city_name)


In [ ]:
incorrect_df['id'] = all_ids
incorrect_df['correct_name'] = all_corrected_names

In [ ]:
incorrect_df.to_csv('./spell_check.csv', index=False)